In [1]:
#!/bin/python3

#%%
import tensorflow as tf
import keras
import numpy as np
import matplotlib.pyplot as plt
import os
import keras_tuner as kt
import sklearn as skl
from sklearn.model_selection import train_test_split

print('Tensorflow version: ' + str(tf.__version__))
print('Keras version: ' + str(keras.__version__))
print('KerasTuner version: ' + str(kt.__version__))
print('Scikit-learn version: ' + str(skl.__version__))

Tensorflow version: 2.15.0
Keras version: 3.0.5
KerasTuner version: 1.4.6
Scikit-learn version: 1.3.0


In [2]:
# load the datasets
path = './datasets/GTSRB/'

print('Loading datasets...')
training = np.load(os.path.join(path, 'training.npz'), allow_pickle=True)
testing = np.load(os.path.join(path, 'testing.npz'), allow_pickle=True)

X_train = training['X']
y_train = training['y']

X_test = testing['X']
y_test = testing['y']

Loading datasets...


In [3]:
# converting datatypes
X_train = np.asarray(X_train).astype('float32')
X_test = np.asarray(X_test).astype('float32')

In [4]:
# split training data into training and validation set
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2,
                                                      shuffle=True)

In [5]:
# augment data

def augment_data(X):
    layers = [
        tf.keras.layers.RandomRotation(1/8), # 45 degrees, either cw/ccw
        tf.keras.layers.RandomTranslation(0.1, 0.1) # shift w/h by up to 10%
    ]

    X_augmented = []

    for x in X:
        for layer in layers:
            x = layer(x)
        
        X_augmented.append(x)
    
    return np.asarray(X_augmented).astype('float32')

print('Augmenting training and validation data...')
X_train = tf.py_function(func=augment_data, inp=[X_train], Tout=tf.float32)
X_valid = tf.py_function(func=augment_data, inp=[X_valid], Tout=tf.float32)

Augmenting training and validation data...


In [6]:
# resizing
print('Resizing...')

X_train = tf.image.resize(X_train, [224, 224], method='nearest')
X_valid = tf.image.resize(X_valid, [224, 224], method='nearest')
X_test = tf.image.resize(X_test, [224, 224], method='nearest')

Resizing...


In [7]:
# build model
print('Searching and building a model based on MobileNet...')

def build_model(hp):
    activation_functions = ['relu', 'sigmoid', 'softmax', 'softplus', 'softsign', 'tanh',
                            'selu', 'elu', 'exponential', 'leaky_relu', 'relu6', 'silu',
                            'gelu', 'hard_sigmoid', 'log_softmax', 'mish', 'linear']
    units_lc0 = hp.Int('units_lc0', 64, 256, 32)
    activation_lc0 = hp.Choice('activation_lc0', activation_functions)
    units_lc1 = hp.Int('units_lc1', 64, 256, 32)
    activation_lc1 = hp.Choice('activation_lc1', activation_functions)
    dropout_l = hp.Float('dropout_l', 0.0, 0.5)
    activation_final = hp.Choice('activation_final', activation_functions)

    mobilenet_model = tf.keras.applications.MobileNet(
        weights="imagenet",
        input_shape=(224, 224, 3),
        include_top=False
    )

    mobilenet_model.trainable = False

    input_layer = tf.keras.layers.Input(shape=(224, 224, 3))
    input_layer = tf.keras.layers.Rescaling(scale=1.0 / 127.5, offset=-1)(input_layer)

    final_model = mobilenet_model(input_layer,
                                  training=False)
    final_model = tf.keras.layers.GlobalMaxPooling2D()(final_model)
    final_model = tf.keras.layers.Flatten()(final_model)
    final_model = tf.keras.layers.Dense(units_lc0, activation=activation_lc0)(final_model)
    final_model = tf.keras.layers.Dense(units_lc1, activation=activation_lc1)(final_model)
    final_model = tf.keras.layers.Dropout(dropout_l)(final_model)
    final_model = tf.keras.layers.Dense(43, activation=activation_final)(final_model) # 43 classes

    final_model = tf.keras.Model(input_layer, final_model)

    loss_from_logits = True
    if activation_final in ['sigmoid', 'softmax']:
        loss_from_logits = False

    final_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=loss_from_logits),
                  metrics=['accuracy'])
    # final_model.summary(show_trainable=True)
    
    return final_model

tuner = kt.BayesianOptimization(build_model,
                        project_name='kt_transfer_learning_model',
                        objective='val_accuracy',
                        max_trials=20,
                        seed=42
                        )
tuner.search(X_train, y_train, epochs=3, validation_data=(X_valid, y_valid), callbacks=[
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=1)
])
tuner.results_summary()
model = tuner.get_best_models()[0]

model.summary(show_trainable=True)

Searching and building a model based on MobileNet...
Reloading Tuner from .\kt_transfer_learning_model\tuner0.json
Results summary
Results in .\kt_transfer_learning_model
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 19 summary
Hyperparameters:
units_lc0: 64
activation_lc0: leaky_relu
units_lc1: 160
activation_lc1: relu6
dropout_l: 0.3630456668613308
activation_final: linear
Score: 0.8009631037712097

Trial 18 summary
Hyperparameters:
units_lc0: 128
activation_lc0: linear
units_lc1: 224
activation_lc1: relu6
dropout_l: 0.07800932022121826
activation_final: softmax
Score: 0.7937399744987488

Trial 02 summary
Hyperparameters:
units_lc0: 256
activation_lc0: sigmoid
units_lc1: 256
activation_lc1: relu
dropout_l: 0.29037858246697834
activation_final: softplus
Score: 0.7580257058143616

Trial 09 summary
Hyperparameters:
units_lc0: 128
activation_lc0: softplus
units_lc1: 128
activation_lc1: selu
dropout_l: 0.07283462755206516
activation_final: leaky_relu
Score:

c:\Users\leona\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 14 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━┓
┃ Layer (type)                ┃ Output Shape          ┃    Param # ┃ Trai… ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━┩
│ input_layer_1 (InputLayer)  │ (None, 224, 224, 3)   │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ rescaling (Rescaling)       │ (None, 224, 224, 3)   │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ mobilenet_1.00_224          │ (None, 7, 7, 1024)    │  3,228,864 │   N   │
│ (Functional)                │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ global_max_pooling2d        │ (None, 1024)          │          0 │   -   │
│ (GlobalMaxPooling2D)        │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ flatten (Flatten)           │ (None, 1024)          │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ dense (Dense)               │ (None, 64)            │     65,600 │   Y   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ dense_1 (Dense)             │ (None, 160)           │     10,400 │   Y   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ dropout (Dropout)           │ (None, 160)           │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ dense_2 (Dense)             │ (None, 43)            │      6,923 │   Y   │
└─────────────────────────────┴───────────────────────┴────────────┴───────┘

 Total params: 3,311,787 (12.63 MB)

 Trainable params: 82,923 (323.92 KB)

 Non-trainable params: 3,228,864 (12.32 MB)

In [8]:
# fit model
print('Fitting model...')
history = model.fit(X_train, y_train, epochs=50, validation_data=(X_valid, y_valid), callbacks=[
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5,
                                     restore_best_weights=True)
])

Fitting model...
Epoch 1/50
312/312 ━━━━━━━━━━━━━━━━━━━━ 123s 381ms/step - accuracy: 0.7205 - loss: 0.8418 - val_accuracy: 0.7986 - val_loss: 0.5445
Epoch 2/50
312/312 ━━━━━━━━━━━━━━━━━━━━ 99s 316ms/step - accuracy: 0.7831 - loss: 0.6198 - val_accuracy: 0.8170 - val_loss: 0.5212
Epoch 3/50
312/312 ━━━━━━━━━━━━━━━━━━━━ 98s 313ms/step - accuracy: 0.8213 - loss: 0.5073 - val_accuracy: 0.8343 - val_loss: 0.4670
Epoch 4/50
312/312 ━━━━━━━━━━━━━━━━━━━━ 98s 315ms/step - accuracy: 0.8508 - loss: 0.4181 - val_accuracy: 0.8523 - val_loss: 0.4116
Epoch 5/50
312/312 ━━━━━━━━━━━━━━━━━━━━ 97s 312ms/step - accuracy: 0.8604 - loss: 0.4056 - val_accuracy: 0.8307 - val_loss: 0.5175
Epoch 6/50
312/312 ━━━━━━━━━━━━━━━━━━━━ 98s 313ms/step - accuracy: 0.8869 - loss: 0.3347 - val_accuracy: 0.8772 - val_loss: 0.3488
Epoch 7/50
312/312 ━━━━━━━━━━━━━━━━━━━━ 89s 286ms/step - accuracy: 0.9017 - loss: 0.2733 - val_accuracy: 0.8832 - val_loss: 0.3607
Epoch 8/50
312/312 ━━━━━━━━━━━━━━━━━━━━ 90s 289ms/step - accuracy

In [9]:
# save model
model.save('transfer_learning_model.keras')

In [10]:
# load model
model = tf.keras.models.load_model('transfer_learning_model.keras')

c:\Users\leona\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'adam', because it has 176 variables whereas the saved optimizer has 14 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


In [11]:
# testing
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print('Test accuracy:', test_acc)

121/121 - 30s - 250ms/step - accuracy: 0.8089 - loss: 0.7008
Test accuracy: 0.8089039325714111
